<a href="https://colab.research.google.com/github/Matthew-Crain/100daysofcode/blob/main/Diabetes_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TASK 1 : Install Dependencies & Run Spark Session

In [1]:
#install pyspark
#install pyspark
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=1f631a91aed014b6ab2f7ef6e4c825449f52f2b8761b706141128fa53805b134
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [6]:
#create a sparksession
#create a sparksession
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("spark").getOrCreate()

# TASK 2: Clone & Explore dataset

In [3]:
#clone the diabetes dataset from the github repository
! git clone https://github.com/education454/diabetes_dataset

Cloning into 'diabetes_dataset'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (6/6), 13.02 KiB | 1.86 MiB/s, done.


In [4]:
#check if the dataset exists
#check if the dataset exists
! ls diabetes_dataset

diabetes.csv  new_test.csv


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
#create spark dataframe
#create spark dataframe
df = spark.read.csv("/content/drive/MyDrive/diabetes_dataset/diabetes.csv",header= True,inferSchema = True)

In [ ]:
#display the dataframe

In [ ]:
#print the schema

In [ ]:
#count the total no. of diabetic and non-diabetic class

In [ ]:
#get the summary statistics

# TASK 3: Data Cleaning & Preparation

In [ ]:
#check for null values

In [ ]:
#look for the unnecessary values present

In [ ]:
#calculate and replace the unnecessary values by the mean value

In [ ]:
#display the dataframe

# TASK 4: Correlation Analysis & Feature Selection

In [ ]:
#find the correlation among the set of input & output variables

In [ ]:
#feature selection

In [ ]:
#print the schema

In [ ]:
#display dataframe

# TASK 5: Split Dataset & Build the Model

In [ ]:
#create final data

In [ ]:
#print schema of final data

In [ ]:
#split the dataset ; build the model

In [ ]:
#summary of the model

# TASK 6: Evaluate and Save the Model

In [ ]:
# save model

In [ ]:
# load saved model back to the environment

# TASK 7: Prediction on New Data with the saved model

In [ ]:
#create a new spark dataframe

In [ ]:
#print the schema

In [ ]:
#create an additional feature merged column

In [ ]:
#print the schema

In [ ]:
#use model to make predictions

In [ ]:
#display the predictions